In [1]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
from typing import Dict, List, Optional, Tuple
from google.colab import files

# plots

In [2]:
# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def safe_filename(filename: str) -> str:
    """Clean filename from special characters."""
    return re.sub(r'[^a-zA-Z0-9\-.]', '', filename)

In [3]:
# ============================================================================
# DATA LOADING AND PROCESSING
# ============================================================================

def load_csv_files(csv_files: List[str]) -> Dict[str, pd.DataFrame]:
    """Load multiple CSV files into a dictionary of DataFrames."""
    data = {}
    for f in csv_files:
        try:
            name = os.path.splitext(os.path.basename(f))[0]
            df = pd.read_csv(f)
            data[name] = df
            print(f"✓ Loaded {name}: {len(df)} rows, {len(df.columns)} columns")
        except Exception as e:
            print(f"✗ Error loading {f}: {e}")
    return data

def extract_numeric_metrics(data: Dict[str, pd.DataFrame]) -> List[str]:
    """Extract all numeric columns that are metrics (excluding epoch and lr)."""
    numeric_cols = {
        col
        for df in data.values()
        for col in df.select_dtypes(include=['float32', 'float64']).columns
    }
    metrics = [
        m for m in numeric_cols
        if 'epoch' not in m.lower() and 'lr' not in m.lower()
    ]
    return sorted(metrics)

def get_final_epoch_metrics(data: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    """Extract metrics from the final epoch for comparison table."""
    final_metrics = []

    for name, df in data.items():
        if 'epoch' in df.columns:
            final_row = df.iloc[-1].copy()  # Last row (final epoch)
            final_row['model'] = name
            final_row['epoch'] = int(final_row['epoch'])
            final_metrics.append(final_row)
        else:
            print(f"Warning: No 'epoch' column found in {name}")

    if not final_metrics:
        return pd.DataFrame()

    # Create comparison table
    comparison_df = pd.DataFrame(final_metrics)

    # Reorder columns: model, epoch, then metrics
    cols = ['model', 'epoch'] + [col for col in comparison_df.columns
                                if col not in ['model', 'epoch']]
    comparison_df = comparison_df[cols]

    return comparison_df

In [4]:
# ============================================================================
# VISUALIZATION
# ============================================================================

def plot_metrics(
    data: Dict[str, pd.DataFrame],
    output_dir: str = '/content/metrics_analysis',
    fmt: str = 'jpg',
    dpi: int = 80,
    figsize: tuple = (8, 4),
    quality: int = 50
) -> None:
    """Create plots for all metrics across models."""
    plots_dir = os.path.join(output_dir, 'plots')
    os.makedirs(plots_dir, exist_ok=True)

    metrics = extract_numeric_metrics(data)

    if not metrics:
        print("No metrics found to plot.")
        return

    print(f"Creating plots for metrics: {', '.join(metrics)}")

    for metric in metrics:
        fig, ax = plt.subplots(figsize=figsize, dpi=dpi)

        for name, df in data.items():
            if metric in df.columns and 'epoch' in df.columns:
                ax.plot(df['epoch'], df[metric], label=name, marker='o', markersize=3)

        ax.set_title(f'{metric} Over Training Epochs')
        ax.set_xlabel('Epoch')
        ax.set_ylabel(metric)
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.tight_layout()

        # Save plot
        fname = safe_filename(metric) + ('.jpg' if fmt.lower() in ('jpg','jpeg') else '.png')
        save_path = os.path.join(plots_dir, fname)

        if fmt.lower() in ('jpg', 'jpeg'):
            fig.savefig(
                save_path,
                format='jpeg',
                dpi=dpi,
                pil_kwargs={'quality': quality, 'optimize': True}
            )
        else:
            fig.savefig(
                save_path,
                format='png',
                dpi=dpi,
                compress_level=9,
                optimize=True
            )

        plt.close(fig)

    print(f"✓ Created {len(metrics)} plots in {plots_dir} (format: {fmt}, dpi: {dpi})")


In [5]:
# ============================================================================
# TABLE GENERATION AND SAVING
# ============================================================================

def save_metrics_table(
    data: Dict[str, pd.DataFrame],
    output_dir: str = '/content/metrics_analysis',
    save_formats: List[str] = ['csv', 'xlsx']
) -> None:
    """Save final epoch metrics in table format for comparison."""
    tables_dir = os.path.join(output_dir, 'tables')
    os.makedirs(tables_dir, exist_ok=True)

    # Get final metrics
    final_metrics = get_final_epoch_metrics(data)

    if final_metrics.empty:
        print("No final metrics to save.")
        return

    # Add validation note
    timestamp = pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')
    base_filename = f'final_validation_metrics_{timestamp}'

    # Save in requested formats
    for fmt in save_formats:
        if fmt.lower() == 'csv':
            filepath = os.path.join(tables_dir, f'{base_filename}.csv')
            final_metrics.to_csv(filepath, index=False)
            print(f"✓ Saved validation metrics table: {filepath}")

        elif fmt.lower() in ['xlsx', 'excel']:
            filepath = os.path.join(tables_dir, f'{base_filename}.xlsx')
            with pd.ExcelWriter(filepath, engine='openpyxl') as writer:
                final_metrics.to_excel(writer, sheet_name='Final_Validation_Metrics', index=False)

                # Add a note sheet
                note_df = pd.DataFrame({
                    'Note': ['These are VALIDATION metrics from the final epoch of training',
                             'NOT test set results',
                             f'Generated on: {pd.Timestamp.now()}',
                             f'Models compared: {", ".join(data.keys())}']
                })
                note_df.to_excel(writer, sheet_name='README', index=False)
            print(f"✓ Saved validation metrics table: {filepath}")

    return final_metrics

def display_metrics_table(data: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    """Display final metrics in a formatted table."""
    final_metrics = get_final_epoch_metrics(data)

    if final_metrics.empty:
        print("No metrics to display.")
        return final_metrics

    print("\n" + "="*80)
    print("FINAL VALIDATION METRICS COMPARISON")
    print("="*80)
    print("Note: These are VALIDATION metrics from the final training epoch")
    print("-"*80)

    # Format for display
    display_df = final_metrics.copy()

    # Round numeric columns to 3 decimal places
    numeric_cols = display_df.select_dtypes(include=['float32', 'float64']).columns
    display_df[numeric_cols] = display_df[numeric_cols].round(3)

    print(display_df.to_string(index=False))
    print("-"*80)

    return final_metrics


In [6]:
# ============================================================================
# MAIN EXECUTION FUNCTIONS
# ============================================================================

def run_full_analysis(
    csv_files: List[str],
    plot_config: Dict = None,
    save_table: bool = True,
    table_formats: List[str] = ['csv', 'xlsx']
) -> Tuple[Dict[str, pd.DataFrame], pd.DataFrame]:
    """Run complete analysis: load data, create plots, and save comparison table."""

    # Default plot configuration
    if plot_config is None:
        plot_config = {
            'output_dir': '/content/metrics_analysis',
            'fmt': 'jpg',
            'dpi': 75,
            'figsize': (20, 10),
            'quality': 75
        }

    print("Starting metrics analysis...")
    print("="*50)

    # 1. Load data
    print("\n1. Loading CSV files...")
    data = load_csv_files(csv_files)

    if not data:
        print("No data loaded. Exiting.")
        return {}, pd.DataFrame()

    # 2. Create plots
    print("\n2. Creating metric plots...")
    plot_metrics(data, **plot_config)

    # 3. Display and save final metrics table
    print("\n3. Processing final validation metrics...")
    final_metrics = display_metrics_table(data)

    if save_table and not final_metrics.empty:
        print("\n4. Saving metrics table...")
        save_metrics_table(data, output_dir=plot_config['output_dir'], save_formats=table_formats)

    print("\n" + "="*50)
    print("Analysis complete!")

    return data, final_metrics


In [7]:
"""Example usage of the metrics analysis tool."""

# Method 1: Upload files in Colab and run full analysis
print("Upload your CSV files:")
uploaded = files.upload()
csv_files = list(uploaded.keys())

# Run complete analysis with custom settings
data, final_metrics = run_full_analysis(
    csv_files,
    plot_config={
        'output_dir': '/content/metrics_analysis',
        'fmt': 'jpg',
        'dpi': 75,
        'figsize': (20, 10),
        'quality': 75
    },
    save_table=True,
    table_formats=['csv', 'xlsx']
)

# Method 2: Step by step analysis
# data = load_csv_files(csv_files)
# plot_metrics(data, fmt='png', dpi=100, figsize=(12, 8))
# final_metrics = display_metrics_table(data)
# save_metrics_table(data, save_formats=['csv'])

Upload your CSV files:


Saving yolo8n_pt_512_coco_balanced_full.csv to yolo8n_pt_512_coco_balanced_full.csv
Saving yolo8n_pt_512_coco_full_training.csv to yolo8n_pt_512_coco_full_training.csv
Saving yolo8n_pt_512_coco_sama_balanced_full.csv to yolo8n_pt_512_coco_sama_balanced_full.csv
Saving yolo8n_pt_512_coco_sama_full_training.csv to yolo8n_pt_512_coco_sama_full_training.csv
Saving yolo8n_pt_512_coco_sama_skiped_crowd.csv to yolo8n_pt_512_coco_sama_skiped_crowd.csv
Saving yolo8n_pt_512_coco_sama_skiped_crowd_3_85.csv to yolo8n_pt_512_coco_sama_skiped_crowd_3_85.csv
Saving yolo8n_pt_512_coco_skiped_crowd.csv to yolo8n_pt_512_coco_skiped_crowd.csv
Saving yolo8n_pt_512_coco_skiped_crowd_2.5_95.csv to yolo8n_pt_512_coco_skiped_crowd_2.5_95.csv
Saving yolo8n_pt_512_coco_skiped_crowd_3_85.csv to yolo8n_pt_512_coco_skiped_crowd_3_85.csv
Starting metrics analysis...

1. Loading CSV files...
✓ Loaded yolo8n_pt_512_coco_balanced_full: 50 rows, 15 columns
✓ Loaded yolo8n_pt_512_coco_full_training: 50 rows, 15 columns


In [8]:
!zip -r /content/metrics_analysis.zip /content/metrics_analysis

  adding: content/metrics_analysis/ (stored 0%)
  adding: content/metrics_analysis/plots/ (stored 0%)
  adding: content/metrics_analysis/plots/valdflloss.jpg (deflated 24%)
  adding: content/metrics_analysis/plots/metricsprecisionB.jpg (deflated 22%)
  adding: content/metrics_analysis/plots/trainboxloss.jpg (deflated 23%)
  adding: content/metrics_analysis/plots/trainclsloss.jpg (deflated 25%)
  adding: content/metrics_analysis/plots/time.jpg (deflated 26%)
  adding: content/metrics_analysis/plots/metricsmAP50B.jpg (deflated 22%)
  adding: content/metrics_analysis/plots/metricsmAP50-95B.jpg (deflated 23%)
  adding: content/metrics_analysis/plots/valboxloss.jpg (deflated 23%)
  adding: content/metrics_analysis/plots/traindflloss.jpg (deflated 24%)
  adding: content/metrics_analysis/plots/metricsrecallB.jpg (deflated 21%)
  adding: content/metrics_analysis/plots/valclsloss.jpg (deflated 24%)
  adding: content/metrics_analysis/tables/ (stored 0%)
  adding: content/metrics_analysis/tables/

In [ ]:
!rm -rf /content/metrics_plots

In [ ]:
example_usage()

Upload your CSV files:


Saving coco_16.csv to coco_16.csv
Saving coco_64.csv to coco_64.csv
Saving coco_128.csv to coco_128.csv
Saving sama_16.csv to sama_16.csv
Saving sama_64.csv to sama_64.csv
Starting metrics analysis...

1. Loading CSV files...
✓ Loaded coco_16: 50 rows, 15 columns
✓ Loaded coco_64: 50 rows, 15 columns
✓ Loaded coco_128: 50 rows, 15 columns
✓ Loaded sama_16: 50 rows, 15 columns
✓ Loaded sama_64: 50 rows, 15 columns

2. Creating metric plots...
Creating plots for metrics: metrics/mAP50(B), metrics/mAP50-95(B), metrics/precision(B), metrics/recall(B), time, train/box_loss, train/cls_loss, train/dfl_loss, val/box_loss, val/cls_loss, val/dfl_loss
✓ Created 11 plots in /content/metrics_analysis/plots (format: jpg, dpi: 75)

3. Processing final validation metrics...

FINAL VALIDATION METRICS COMPARISON
Note: These are VALIDATION metrics from the final training epoch
--------------------------------------------------------------------------------
   model  epoch     time  train/box_loss  train/

# videos

In [ ]:
!pip install ultralytics sahi lap

In [ ]:
import os
import torch
from ultralytics import YOLO
import cv2
from tqdm import tqdm
import shutil
from google.colab import files
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Створення основних директорій
base_dirs = [
    '/content/models',  # Директорія для моделей .pt
    '/content/videos',  # Директорія для вихідних відео
    '/content/results'  # Директорія для результатів
]

for dir_path in base_dirs:
    os.makedirs(dir_path, exist_ok=True)
    print(f"Created directory: {dir_path}")

print("\nПапки створено. Тепер ви можете вручну закинути:")
print("1. Моделі .pt в /content/models")
print("2. Відео в /content/videos")
print("3. Результати будуть в /content/results")

Created directory: /content/models
Created directory: /content/videos
Created directory: /content/results

Папки створено. Тепер ви можете вручну закинути:
1. Моделі .pt в /content/models
2. Відео в /content/videos
3. Результати будуть в /content/results


In [ ]:
class VideoTrackingConfig:
    def __init__(self):
        """
        Конфігурація гіперпараметрів для трекінгу об'єктів у відео
        """
        # Параметри трекінгу/детекції
        self.CONFIDENCE_THRESHOLD = 0.7
        self.IOU_THRESHOLD = 0.6
        self.MAX_DETECTIONS = 300
        self.AGNOSTIC_NMS = False
        self.SELECTED_CLASSES = None

        # Візуалізація
        self.DRAW_LABELS = True
        self.DRAW_CONFIDENCE = True
        self.BBOX_THICKNESS = 2
        self.LABEL_FONT_SCALE = 0.7

        # Відео вивід
        self.OUTPUT_CODEC = 'mp4v'
        self.RESIZE_OUTPUT = None  # (width, height)

        # Трекінг
        self.TRACKING = True  # Увімкнути трекінг
        self.PERSIST_LABELS = True  # Покращене збереження ID треків на об'єктах


def main_tracking(config=None):
    """
    Основна функція для трекінгу об'єктів у відео
    """
    import os
    from ultralytics import YOLO
    import cv2

    if config is None:
        config = VideoTrackingConfig()

    MODELS_DIR = '/content/models'
    VIDEOS_DIR = '/content/videos'
    RESULTS_DIR = '/content/results'

    model_files = [f for f in os.listdir(MODELS_DIR) if f.endswith('.pt')]
    video_files = [f for f in os.listdir(VIDEOS_DIR) if f.endswith(('.mp4', '.avi', '.mov', '.mkv'))]

    for model_file in model_files:
        model_name = os.path.splitext(model_file)[0]
        model_results_dir = os.path.join(RESULTS_DIR, model_name)
        os.makedirs(model_results_dir, exist_ok=True)

        model = YOLO(os.path.join(MODELS_DIR, model_file))

        model.overrides.update({
            'conf': config.CONFIDENCE_THRESHOLD,
            'iou': config.IOU_THRESHOLD,
            'max_det': config.MAX_DETECTIONS,
            'agnostic_nms': config.AGNOSTIC_NMS,
            'classes': config.SELECTED_CLASSES
        })


        for video_file in video_files:
            input_path = os.path.join(VIDEOS_DIR, video_file)
            output_path = os.path.join(model_results_dir, video_file)

            print(f"Tracking {model_name}: {video_file}")

            try:
                cap = cv2.VideoCapture(input_path)
                if not cap.isOpened():
                    print(f"Error: Cannot open video file {input_path}")
                    continue

                total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                fps = cap.get(cv2.CAP_PROP_FPS)

                if config.RESIZE_OUTPUT:
                    width, height = config.RESIZE_OUTPUT

                fourcc = cv2.VideoWriter_fourcc(*config.OUTPUT_CODEC)
                out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

                cap.release()


                model.tracker = None

                results = model.track(
                    source=input_path,
                    stream=True,
                    persist=config.PERSIST_LABELS,
                    conf=config.CONFIDENCE_THRESHOLD,
                    iou=config.IOU_THRESHOLD,
                    classes=config.SELECTED_CLASSES,
                    verbose=False,
                )

                for result in tqdm(results, total=total_frames, desc=f"  ▶ {video_file}"):
                    annotated_frame = result.plot(
                        conf=config.DRAW_CONFIDENCE,
                        labels=config.DRAW_LABELS,
                        font_size=config.LABEL_FONT_SCALE,
                        line_width=config.BBOX_THICKNESS
                    )

                    if config.RESIZE_OUTPUT:
                        annotated_frame = cv2.resize(annotated_frame, config.RESIZE_OUTPUT)

                    out.write(annotated_frame)

                out.release()
                print(f"  ✅ Completed: {video_file}")

            except Exception as e:
                print(f"Error tracking {video_file}: {str(e)}")



# Виклик з кастомною конфігурацією
custom_config = VideoTrackingConfig()
main_tracking(custom_config)

Tracking yolo8n_pt_512_coco_balanced_full: 535.mp4


  ▶ 535.mp4: 100%|██████████| 619/619 [00:35<00:00, 17.22it/s]


  ✅ Completed: 535.mp4
Tracking yolo8n_pt_512_coco_balanced_full: dogs.mp4


  ▶ dogs.mp4: 100%|██████████| 1641/1641 [00:27<00:00, 59.70it/s]


  ✅ Completed: dogs.mp4
Tracking yolo8n_pt_512_coco_balanced_full: Shibuya.mp4


...

  ✅ Completed: 508.mp4
Tracking yolo8n_pt_512_coco_sama_balanced_full: ring_doorbell.mp4


  ▶ ring_doorbell.mp4: 100%|█████████▉| 2953/2956 [01:08<00:00, 42.98it/s]


  ✅ Completed: ring_doorbell.mp4
Tracking yolo8n_pt_512_coco_sama_balanced_full: persons.mp4


  ▶ persons.mp4: 100%|██████████| 1501/1501 [00:26<00:00, 55.78it/s]

  ✅ Completed: persons.mp4


In [ ]:
!zip -r /content/results.zip /content/results

  adding: content/results/ (stored 0%)
  adding: content/results/yolo8n_pt_512_coco_skiped_crowd/ (stored 0%)
  adding: content/results/yolo8n_pt_512_coco_skiped_crowd/535.mp4 (deflated 0%)
  adding: content/results/yolo8n_pt_512_coco_skiped_crowd/dogs.mp4 (deflated 2%)
  adding: content/results/yolo8n_pt_512_coco_skiped_crowd/Shibuya.mp4 (deflated 2%)
  adding: content/results/yolo8n_pt_512_coco_skiped_crowd/508.mp4 (deflated 4%)
  adding: content/results/yolo8n_pt_512_coco_skiped_crowd/ring_doorbell.mp4 (deflated 7%)
  adding: content/results/yolo8n_pt_512_coco_skiped_crowd/persons.mp4 (deflated 3%)
  adding: content/results/yolo8n_pt_512_coco_sama_skiped_crowd_3_85/ (stored 0%)
  adding: content/results/yolo8n_pt_512_coco_sama_skiped_crowd_3_85/535.mp4 (deflated 0%)
  adding: content/results/yolo8n_pt_512_coco_sama_skiped_crowd_3_85/dogs.mp4 (deflated 2%)
  adding: content/results/yolo8n_pt_512_coco_sama_skiped_crowd_3_85/Shibuya.mp4 (deflated 2%)
  adding: content/results/yolo8n_pt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mv /content/results.zip /content/drive/MyDrive